In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
from os import listdir
from os.path import isfile, join
import seaborn as sns
from multiprocessing import Pool

In [4]:
oil = pd.read_csv("data/oil.csv")
stores = pd.read_csv("data/stores.csv")
holidays_events = pd.read_csv("data/holidays_events.csv")
transactions = pd.read_csv("data/transactions.csv")
items = pd.read_csv("data/items.csv")
mainTrain = pd.read_pickle("trainArray5Mil.pickle")
mainTest = pd.read_pickle("testArray5Mil.pickle")

In [18]:
mainTrain["dcoilwtico"] = mainTrain["dcoilwtico"].fillna(method="pad")
mainTrain.head()
# print(mainTrain.isnull().sum())

,id,date,store_nbr,item_nbr,unit_sales,onpromotion,city,state,type,cluster,transactions,dcoilwtico,family,class,perishable,month,year,day
0,22000000,2014-04-07,39,3958,38.0,False,41,41,38,38,1447,100.43,15,131,0,4,2014,7
1,22000001,2014-04-07,39,703,1.0,False,41,41,38,38,1447,100.43,15,131,0,4,2014,7
2,22000002,2014-04-07,39,1124,4.0,False,41,41,38,38,1447,100.43,5,93,1,4,2014,7
3,22000003,2014-04-07,39,2024,11.0,False,41,41,38,38,1447,100.43,15,163,0,4,2014,7
4,22000004,2014-04-07,39,264,4.0,False,41,41,38,38,1447,100.43,24,225,0,4,2014,7


In [ ]:
store_nbr = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("store_nbr", hash_bucket_size=54), dimension=3)
onpromotion = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("onpromotion", hash_bucket_size=2), dimension=3)
type = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("type", hash_bucket_size=2), dimension=3)
family = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("family", hash_bucket_size=2), dimension=3)
classCol = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("class", hash_bucket_size=2), dimension=3)
perishable = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("perishable", hash_bucket_size=2), dimension=3)
month = tf.contrib.layers.embedding_column(tf.contrib.layers.sparse_column_with_hash_bucket("month", hash_bucket_size=2), dimension=3)